In [ ]:
import os
import shutil
import random
from google.colab import drive  # Import the drive module

# Mount Google Drive. This will prompt you for authorization.
drive.mount('/content/drive')

# Define the path to the dataset
dataset_path = "/content/drive/MyDrive/ArSL dataset/ArASL_Database_54K_Final"  # Replace with your actual path


Mounted at /content/drive


In [ ]:
import os
import shutil
import random
from google.colab import drive  # Import the drive module

# Mount Google Drive. This will prompt you for authorization.
drive.mount('/content/drive')

# Define the path to the dataset
dataset_path = "/content/drive/MyDrive/ArSL dataset/ArASL_Database_54K_Final"  # Replace with your actual path

# Initialize a dictionary to store the image counts for each folder
folder_image_counts = {}

# Iterate over the subfolders within the dataset directory
for folder_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder_name)

    # Check if it's a directory (to avoid counting any stray files)
    if os.path.isdir(folder_path):
        # Count the number of files in the subfolder
        image_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]) #Count the number of files

        # Store the image count in the dictionary
        folder_image_counts[folder_name] = image_count

# Print the image counts for each folder
for folder, count in folder_image_counts.items():
    print(f"Folder: {folder}, Image Count: {count}")


# Calculate the total number of images
total_images = sum(folder_image_counts.values())
print(f"\nTotal Number of Images: {total_images}")



ValueError: Mountpoint must not already contain files

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

dataset_path = "/content/drive/MyDrive/ArSL dataset/ArASL_Database_54K_Final"  # DOUBLE-CHECK THIS

# Print the list of folders found in the dataset path
print("Folders found:", os.listdir(dataset_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folders found: ['ain', 'al', 'aleff', 'bb', 'dal', 'dha', 'dhad', 'fa', 'gaaf', 'ghain', 'ha', 'haa', 'jeem', 'kaaf', 'khaa', 'la', 'laam', 'meem', 'nun', 'ra', 'saad', 'seen', 'sheen', 'ta', 'taa', 'thaa', 'thal', 'toot', 'waw', 'ya', 'yaa', 'zay']


In [ ]:
import os
import shutil
import random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

def create_train_test_split(source_dir, train_dir, test_dir, train_ratio=0.7, batch_size=32):
    # Create output directories
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Data transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    for class_name in os.listdir(source_dir):
        class_dir = os.path.join(source_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

        images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        random.shuffle(images)

        split_idx = int(len(images) * train_ratio)
        train_images = images[:split_idx]
        test_images = images[split_idx:]

        # Copy with GPU acceleration if possible
        for img in train_images:
            shutil.copy2(
                os.path.join(class_dir, img),
                os.path.join(train_dir, class_name, img)
            )

        for img in test_images:
            shutil.copy2(
                os.path.join(class_dir, img),
                os.path.join(test_dir, class_name, img)
            )

        print(f"Class {class_name}: {len(train_images)} train, {len(test_images)} test")

    # Create DataLoaders
    train_dataset = datasets.ImageFolder(train_dir, transform=transform)
    test_dataset = datasets.ImageFolder(test_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

    return train_loader, test_loader

# Usage
source_dir = "/content/drive/MyDrive/ArSL dataset/ArASL_Database_54K_Final"
train_dir = "/content/drive/MyDrive/ArSL dataset/train"
test_dir = "/content/drive/MyDrive/ArSL dataset/test"

train_loader, test_loader = create_train_test_split(source_dir, train_dir, test_dir)

Using device: cuda
Class ain: 1479 train, 635 test
Class al: 940 train, 403 test
Class aleff: 1170 train, 502 test
Class bb: 1253 train, 538 test
Class dal: 1143 train, 491 test
Class dha: 1206 train, 517 test
Class dhad: 1169 train, 501 test
Class fa: 1368 train, 587 test
Class gaaf: 1193 train, 512 test
Class ghain: 1383 train, 594 test
Class ha: 1114 train, 478 test
Class haa: 1068 train, 458 test
Class jeem: 1086 train, 466 test
Class kaaf: 1241 train, 533 test
Class khaa: 1124 train, 483 test
Class la: 1222 train, 524 test
Class laam: 1282 train, 550 test
Class meem: 1235 train, 530 test


KeyboardInterrupt: 

In [ ]:
import os
import shutil
import random
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def copy_images(args):
    src_path, dst_path, images = args
    for img in images:
        try:
            shutil.copy2(
                os.path.join(src_path, img),
                os.path.join(dst_path, img)
            )
        except Exception as e:
            print(f"Error copying {img}: {str(e)}")

def continue_train_test_split(source_dir, train_dir, test_dir, start_after="meem", train_ratio=0.7, num_workers=4):
    print(f"Using {num_workers} workers for parallel processing")

    # Create output directories
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Get list of all class folders
    class_folders = [f for f in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, f))]

    # Find the index to start from
    try:
        start_idx = class_folders.index(start_after) + 1
        class_folders = class_folders[start_idx:]
        print(f"Continuing from after {start_after}, {len(class_folders)} classes remaining")
    except ValueError:
        print(f"Warning: {start_after} not found, processing all classes")

    # Process each remaining class with progress bar
    for class_name in tqdm(class_folders, desc="Processing remaining classes"):
        try:
            class_dir = os.path.join(source_dir, class_name)
            train_class_dir = os.path.join(train_dir, class_name)
            test_class_dir = os.path.join(test_dir, class_name)

            # Create class directories
            os.makedirs(train_class_dir, exist_ok=True)
            os.makedirs(test_class_dir, exist_ok=True)

            # Get all images
            images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
            random.shuffle(images)

            # Split indices
            split_idx = int(len(images) * train_ratio)
            train_images = images[:split_idx]
            test_images = images[split_idx:]

            # Parallel copy using ThreadPoolExecutor
            with ThreadPoolExecutor(max_workers=num_workers) as executor:
                # Copy train images
                chunk_size = max(1, len(train_images) // num_workers)
                chunks = [train_images[i:i + chunk_size] for i in range(0, len(train_images), chunk_size)]
                args = [(class_dir, train_class_dir, chunk) for chunk in chunks]
                executor.map(copy_images, args)

                # Copy test images
                chunk_size = max(1, len(test_images) // num_workers)
                chunks = [test_images[i:i + chunk_size] for i in range(0, len(test_images), chunk_size)]
                args = [(class_dir, test_class_dir, chunk) for chunk in chunks]
                executor.map(copy_images, args)

            print(f"Class {class_name}: {len(train_images)} train, {len(test_images)} test")

        except Exception as e:
            print(f"Error processing class {class_name}: {str(e)}")
            continue

if __name__ == "__main__":
    # Paths
    source_dir = "/content/drive/MyDrive/ArSL dataset/ArASL_Database_54K_Final"
    train_dir = "/content/drive/MyDrive/ArSL dataset/train"
    test_dir = "/content/drive/MyDrive/ArSL dataset/test"

    # Continue the split from after 'meem'
    continue_train_test_split(source_dir, train_dir, test_dir, start_after="meem")

    print("Remaining split complete!")

Using 4 workers for parallel processing
Continuing from after meem, 14 classes remaining


Processing remaining classes:   7%|▋         | 1/14 [00:38<08:18, 38.36s/it]

Class nun: 1273 train, 546 test


Processing remaining classes:  14%|█▍        | 2/14 [01:40<10:24, 52.07s/it]

Class ra: 1161 train, 498 test


...

Class ya: 1205 train, 517 test


Processing remaining classes:  93%|█████████▎| 13/14 [12:47<00:54, 54.08s/it]

Class yaa: 905 train, 388 test


Processing remaining classes: 100%|██████████| 14/14 [13:45<00:00, 58.96s/it]

Class zay: 961 train, 413 test
Remaining split complete!
